In [4]:
import lightning as pl

In [10]:
import lightning as L

In [11]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning.pytorch as pl

# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))


# define the LightningModule
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)


In [12]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

100%|██████████| 9912422/9912422 [00:03<00:00, 3292977.49it/s]


Extracting /workspaces/pytorch-sandbox/hello-world/hello_world/MNIST/raw/train-images-idx3-ubyte.gz to /workspaces/pytorch-sandbox/hello-world/hello_world/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 60176698.37it/s]


Extracting /workspaces/pytorch-sandbox/hello-world/hello_world/MNIST/raw/train-labels-idx1-ubyte.gz to /workspaces/pytorch-sandbox/hello-world/hello_world/MNIST/raw



100%|██████████| 1648877/1648877 [00:01<00:00, 981682.23it/s] 


Extracting /workspaces/pytorch-sandbox/hello-world/hello_world/MNIST/raw/t10k-images-idx3-ubyte.gz to /workspaces/pytorch-sandbox/hello-world/hello_world/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno -3] Temporary failure in name resolution>



100%|██████████| 4542/4542 [00:00<00:00, 5894346.77it/s]

Extracting /workspaces/pytorch-sandbox/hello-world/hello_world/MNIST/raw/t10k-labels-idx1-ubyte.gz to /workspaces/pytorch-sandbox/hello-world/hello_world/MNIST/raw



In [13]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/workspaces/pytorch-sandbox/hello-world/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total pa

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [14]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = Tensor(4, 28 * 28)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[-3.1154e+21,  1.6525e+21,  5.6900e+21],
        [-1.3598e+06, -1.8423e+05,  2.9735e+06],
        [-1.5419e+06, -7.8563e+05,  2.7908e+06],
        [-4.8115e+05, -1.0524e+06,  3.3586e+05]], grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡


In [18]:
tensorboard --logdir

NameError: name 'tensorboard' is not defined